In [1]:
#importing packages necessary to read file, construct visualization of data and for analysis 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import statsmodels.formula.api as smf 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression, ARDRegression

In [2]:
#importing the excel file under new name and reading it
file = './birthweight_low.xlsx'
bw = pd.read_excel(io = file,
                   sheet_name = 0,
                   header = 0)

#displaying first ten rows of dataframe
bw.head(n=10)

,mage,meduc,monpre,npvis,fage,feduc,omaps,fmaps,cigs,drink,male,mwhte,mblck,moth,fwhte,fblck,foth,bwght
0,71,14.0,4,7.0,51,11.0,9,8,15,13,0,1,0,0,1,0,0,2068
1,71,12.0,3,6.0,46,12.0,2,7,21,12,1,0,1,0,0,1,0,1490
2,70,14.0,3,9.0,59,16.0,8,9,8,10,0,0,0,1,0,0,1,2948
3,54,12.0,2,12.0,46,12.0,9,9,17,12,1,0,1,0,0,1,0,2050
4,68,12.0,3,10.0,61,11.0,4,6,25,11,1,1,0,0,1,0,0,1290
5,67,11.0,4,8.0,40,8.0,4,9,16,14,0,1,0,0,1,0,0,1984
6,64,12.0,1,35.0,49,10.0,9,10,10,4,0,1,0,0,1,0,0,2890
7,40,14.0,1,12.0,36,14.0,9,9,16,10,1,0,1,0,0,1,0,2580
8,62,14.0,3,12.0,54,14.0,9,9,16,7,1,1,0,0,1,0,0,2939
9,61,17.0,2,12.0,73,17.0,7,8,17,9,0,0,0,1,0,0,1,2980


In [3]:
#flaggin missing values

# creating a loop to identify the features with missing values and create a column with m_ identificator
# marking a 0 if value is not missing and 1 if the value is missing
for col in bw:
    if bw[col].isnull().astype(int).sum(axis=0) > 0:
        bw['m_'+col] = bw[col].isnull().astype(int)
        
#checking loop works properly by summing values in the new columns 
bw[['m_meduc','m_npvis','m_feduc']].sum(axis=0)

m_meduc    3
m_npvis    3
m_feduc    7
dtype: int64

In [4]:
#imputing missing values for features using mean
meduc_mean = bw['meduc'].mean() #setting mean into variable
bw['meduc'] = bw['meduc'].fillna(value = meduc_mean) #using variable to fill over same column

npvis_mean = bw['npvis'].mean()
bw['npvis'] = bw['npvis'].fillna(value = npvis_mean)

feduc_mean = bw['feduc'].mean()
bw['feduc'] = bw['feduc'].fillna(value = feduc_mean)

#dropping omaps and fmaps, since they happen after birth, they are not explanatory
bw = bw.drop('omaps', axis=1)
bw = bw.drop('fmaps', axis=1)

bw.head(n=10)

,mage,meduc,monpre,npvis,fage,feduc,cigs,drink,male,mwhte,mblck,moth,fwhte,fblck,foth,bwght,m_meduc,m_npvis,m_feduc
0,71,14.0,4,7.0,51,11.0,15,13,0,1,0,0,1,0,0,2068,0,0,0
1,71,12.0,3,6.0,46,12.0,21,12,1,0,1,0,0,1,0,1490,0,0,0
2,70,14.0,3,9.0,59,16.0,8,10,0,0,0,1,0,0,1,2948,0,0,0
3,54,12.0,2,12.0,46,12.0,17,12,1,0,1,0,0,1,0,2050,0,0,0
4,68,12.0,3,10.0,61,11.0,25,11,1,1,0,0,1,0,0,1290,0,0,0
5,67,11.0,4,8.0,40,8.0,16,14,0,1,0,0,1,0,0,1984,0,0,0
6,64,12.0,1,35.0,49,10.0,10,4,0,1,0,0,1,0,0,2890,0,0,0
7,40,14.0,1,12.0,36,14.0,16,10,1,0,1,0,0,1,0,2580,0,0,0
8,62,14.0,3,12.0,54,14.0,16,7,1,1,0,0,1,0,0,2939,0,0,0
9,61,17.0,2,12.0,73,17.0,17,9,0,0,0,1,0,0,1,2980,0,0,0


In [5]:
#printing perason correlations for birthweight
bw_corr = bw.corr().round(decimals=2) #setting correlation
print(bw_corr.loc['bwght'].sort_values(ascending = False))

bwght      1.00
feduc      0.13
mblck      0.13
fblck      0.12
male       0.11
meduc      0.09
npvis      0.06
m_npvis    0.06
m_feduc   -0.00
moth      -0.02
fwhte     -0.04
monpre    -0.05
foth      -0.08
mwhte     -0.11
m_meduc   -0.13
fage      -0.40
mage      -0.46
cigs      -0.57
drink     -0.74
Name: bwght, dtype: float64


Tried to create different bins for the non dummy variables given, due to the fact that after creating scatter plot for each of them they appeared to be Interval/Count data. 
The bins created looked as follows:

cigarettes = 0-5, 6-10, 11-15, 16 or more 
drinks = 0-4 (normal), 5-8 (moderate), 9 or more (heavy)
ages = 20-30 (twenties), 31-40 (thirties), 41-50 (forties) and above fifty 
month prenatal = 1-3 (first trimester), 4-6 (second trimester), 7-9 (third trimester)
npvis = as is average (between 10-15)
education = 0-12 (Basic Education), 13-15 (Higher Education) and 16 above (Post or phd)

in the cases of some of them like monpre or npvis, the correlations given after the creation of bins and dummies canceled each other out. 

Tried to do the logarithmic for y but when running on the models scores came out as 0s.

In [6]:
#creating copy of original data to modify 
bw_c = pd.DataFrame.copy(bw)

#multiplication variables
bw_c['prnts_ave_ag'] = ((bw['mage'] + bw['fage'])/2)
bw_c['mage_cigs'] = bw['mage']*bw['cigs']
bw_c['mage_drink'] = bw['mage']*bw['drink']


#bins for number of prenatal visits (10-15 is considered average)        
bw_c['npvis'] = np.where((bw['npvis'] >= 10) & (bw['npvis'] <= 15), 1,0)

#other combinations of variables that didn't work ############################

#bw_c['prnts_ave_educ'] = ((bw['meduc'] + bw['feduc'])/2)
#bw_c['vis_per_month'] = bw['npvis'] / bw['monpre']
#bw_c['drinks'] = np.where((bw['drink'] >= 4), 1,0) #recommended to keep consumption to 1-4 units per week
#bw_c['smokes'] = np.where((bw['cigs'] > 0), 1,0) #recomended not to smoke at all 
#bw_c['prnts_whte'] = np.where((bw['mwhte'] == 1) & (bw['fwhte'] == 1), 1,0)
#bw_c['prnts_blck'] = np.where((bw['mblck'] == 1) & (bw['fblck'] == 1), 1,0)
#bw_c['prnts_mix'] = np.where((bw_c['prnts_whte'] == 0) & (bw_c['prnts_blck'] == 0), 1,0)


bw_c.head(n=10)

,mage,meduc,monpre,npvis,fage,feduc,cigs,drink,male,mwhte,...,fwhte,fblck,foth,bwght,m_meduc,m_npvis,m_feduc,prnts_ave_ag,mage_cigs,mage_drink
0,71,14.0,4,0,51,11.0,15,13,0,1,...,1,0,0,2068,0,0,0,61.0,1065,923
1,71,12.0,3,0,46,12.0,21,12,1,0,...,0,1,0,1490,0,0,0,58.5,1491,852
2,70,14.0,3,0,59,16.0,8,10,0,0,...,0,0,1,2948,0,0,0,64.5,560,700
3,54,12.0,2,1,46,12.0,17,12,1,0,...,0,1,0,2050,0,0,0,50.0,918,648
4,68,12.0,3,1,61,11.0,25,11,1,1,...,1,0,0,1290,0,0,0,64.5,1700,748
5,67,11.0,4,0,40,8.0,16,14,0,1,...,1,0,0,1984,0,0,0,53.5,1072,938
6,64,12.0,1,0,49,10.0,10,4,0,1,...,1,0,0,2890,0,0,0,56.5,640,256
7,40,14.0,1,1,36,14.0,16,10,1,0,...,0,1,0,2580,0,0,0,38.0,640,400
8,62,14.0,3,1,54,14.0,16,7,1,1,...,1,0,0,2939,0,0,0,58.0,992,434
9,61,17.0,2,1,73,17.0,17,9,0,0,...,0,0,1,2980,0,0,0,67.0,1037,549


In [7]:
#checking pearson correlation for new DataFrame 
bw_corr = bw_c.corr(method='pearson').round(decimals=2)
bw_corr['bwght']

mage           -0.46
meduc           0.09
monpre         -0.05
npvis           0.15
fage           -0.40
feduc           0.13
cigs           -0.57
drink          -0.74
male            0.11
mwhte          -0.11
mblck           0.13
moth           -0.02
fwhte          -0.04
fblck           0.12
foth           -0.08
bwght           1.00
m_meduc        -0.13
m_npvis         0.06
m_feduc        -0.00
prnts_ave_ag   -0.49
mage_cigs      -0.68
mage_drink     -0.77
Name: bwght, dtype: float64

Based from the correlations above, we can see that the only variables with a moderate/ high correlation to birthweight are those related to age, drinking or smoking and missing values.

In [8]:
#running an initial OLS regression
lm_bw_c = smf.ols(formula = """bwght ~ cigs+
drink+
male+
m_npvis+
prnts_ave_ag""",
                               data = bw_c)


# telling Python to run the data through the model
results_bw_c = lm_bw_c.fit()


# printing the results of ols model
results_bw_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  bwght   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     93.27
Date:                Wed, 24 Nov 2021   Prob (F-statistic):           3.05e-49
Time:                        19:40:54   Log-Likelihood:                -1424.6
No. Observations:                 196   AIC:                             2861.
Df Residuals:                     190   BIC:                             2881.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     5031.4103    128.194     39.248      0.000    4778.543    5284.277
cigs           -36.1879      4.477     -8.083      0.000     -45.019     -27.357
drink         -116.1386      9.488    -12.240      0.000    -134.855     -97.422
male             1.9429     51.434      0.038      0.970     -99.513     103.398
m_npvis        342.3629    207.448      1.650      0.101     -66.835     751.560
prnts_ave_ag   -17.1502      3.165     -5.419      0.000     -23.393     -10.908
==============================================================================
Omnibus:                        6.753   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.034   Jarque-Bera (JB):               11.313
Skew:                           0.011   Prob(JB):                      0.00349
Kurtosis:                       4.177   Cond. No.                         350.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
#setting into a variable the features from initial OLS 
x_variables = ['cigs', 'drink', 'male', 'm_npvis',
               'prnts_ave_ag']

#preparing explanatory and response variables 

bw_data = bw_c.loc[ : , x_variables]
bw_target = bw_c.loc[ : ,'bwght']

#split the given dataset into train and test sets (using lowercase x)
x_train,x_test,y_train,y_test = train_test_split(bw_data,
                                                 bw_target,
                                                 test_size=0.25,
                                                 random_state=219)

In [10]:
# instantiating lasso model
lasso_model = Lasso(alpha = 1.0,
                    normalize = True) # default magitude

# fitting the model to trainning data 
lasso_fit = lasso_model.fit(x_train, y_train)


# predicting y based on the new test data
lasso_pred = lasso_fit.predict(x_test)


# printing the scores (Rsquared) for both models 
print('Lasso Training Score :', lasso_model.score(x_train, y_train).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test, y_test).round(4))


# saving scoring data for template
lasso_train_score = lasso_model.score(x_train, y_train).round(4) #R-square
lasso_test_score  = lasso_model.score(x_test, y_test).round(4)   #R-square


# Calculating gap between trained and test model and saving it for future use
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)



Lasso Training Score : 0.709
Lasso Testing Score  : 0.7074
Lasso Train-Test Gap : 0.0016


In [11]:
#setting into variables the output generated from lasso regression to run Linar Regression
# x-variables
lasso_data = bw_c.loc[ : , ['cigs','drink','m_npvis','prnts_ave_ag']]
# y-variable
lasso_target = bw.loc[ : , 'bwght']

lasso_x_train, lasso_x_test, lasso_y_train, lasso_y_test = train_test_split(
            lasso_data,     # x-variables
            lasso_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

In [12]:
# instantiating Linear Regression Model
lr = LinearRegression()


# fitting the model to trainning data 
lr_fit = lr.fit(lasso_x_train, lasso_y_train)


# predicting y based on the new test data
lr_pred = lr_fit.predict(lasso_x_test)


# printing the scores (Rsquared) for both models 
print('OLS Training Score :', lr.score(lasso_x_train, lasso_y_train).round(4))  #R-square
print('OLS Testing Score  :',  lr.score(lasso_x_test, lasso_y_test).round(4)) #R-square

# saving scoring data for template
lr_train_score = lr.score(lasso_x_train, lasso_y_train).round(4)
lr_test_score = lr.score(lasso_x_test, lasso_y_test).round(4)


# Calculating gap between trained and test model and saving it for future use
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

OLS Training Score : 0.7098
OLS Testing Score  : 0.7089
OLS Train-Test Gap : 0.0009


In [13]:
# instantiating ARD Regression Model
ard_model = ARDRegression()


# fitting the model to trainning data (same variables given to Lasso)
ard_fit = ard_model.fit(x_train, y_train)


# printing the scores (Rsquared) for both models 
ard_pred = ard_model.predict(x_test)

# printing the scores (Rsquared) for both models 
print('Training Score:', ard_model.score(x_train,y_train).round(decimals=4)) #R-square
print('Testing Score :',  ard_model.score(x_test,y_test).round(decimals=4)) #R-square


# saving scoring data for template
ard_train_score = ard_model.score(x_train,y_train).round(decimals=4)
ard_test_score  = ard_model.score(x_test,y_test).round(decimals=4)


# Calculating gap between trained and test model and saving it for future use
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

Training Score: 0.7086
Testing Score : 0.7066
ARD Train-Test Gap : 0.002


In [14]:
#Comparison of testing gaps for the different models 

print(f"""
Model      Train Score      Test Score     Gap
-----      -----------      ----------     ----------
*OLS       {lr_train_score}           {lr_test_score}         {lr_test_gap}
Lasso      {lasso_train_score}            {lasso_test_score}         {lasso_test_gap}
ARD        {ard_train_score}           {ard_test_score}         {ard_test_gap}

* best model = OLS

""")


Model      Train Score      Test Score     Gap
-----      -----------      ----------     ----------
*OLS       0.7098           0.7089         0.0009
Lasso      0.709            0.7074         0.0016
ARD        0.7086           0.7066         0.002

* best model = OLS




References:
Code taken from course material.

links from which information was looked to construct bins:

https://www.whattoexpect.com/pregnancy/pregnancy-health/prenatal-appointments/
https://utswmed.org/medblog/alcohol-during-pregnancy/
https://www.healthline.com/health/pregnancy/best-age-to-get-pregnant#get-help
https://utswmed.org/medblog/alcohol-during-pregnancy/
